# Create Master DF for plottin 
Purpose: takes all Strategic environmental assessment output csvs and merges them into one large table in the long format for plotting

In [1]:
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
#plotly.tools.set_credentials_file(username='grace.cc.wu', api_key='mfClrA9eHWhuOwBJLR3h')
import pandas as pd
import os
import math
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
matplotlib.style.use('ggplot')
plt.style.use('ggplot')
pd.set_option('display.width', 1000)

## Preprocess input CSVs

### Function: pre-process envData csv (to use inside a loop)

In [14]:
def processCSV(csv, gaplc_df):
    selSite_df = pd.read_csv(csv, keep_default_na=False)

    ''' ### ADD FLAG COLUMN, impactEval ### '''
    ## add new column for impactEval (1 = an impact assessment was performed for the env categories)
    selSite_df["impactEval"] = 1
    ## Where the area_envData_km2 column was not calculated (empty feature classes), then set value to 0 (applies to jupyter notebook anayses)
    selSite_df.loc[selSite_df.area_envData_km2.astype(str) == "NA", 'impactEval'] = 0

    ''' ### SUBSET GAPLC TO TECHNOLOGIES IN THE CSV ### '''
    techList = selSite_df["tech"].unique()
    gaplc_df_sub = gaplc_df[(gaplc_df["tech"].isin(techList))]

    ''' ### MERGE CSV WITH GAPLC ### '''
    ## merge selSite_df with the gaplc df to get all the "area_allSelSites_km2" and "selSiteExist"
    selSite_merged_df = selSite_df.merge(gaplc_df_sub[["selSites", "region", "selSitesExist", "area_allSelSites_km2", "envCat", "tech"]], how = "outer", on =  ["selSites", "region"])

    ''' ### ADD INFRASTRUCTURE COLUMN ### '''
    if "selSite" in csv:
        selSite_merged_df["infrastructure"] = "generation"
    if "tx" in csv:
        selSite_merged_df["infrastructure"] = "transmission"

    ''' ### EXPAND envData COLUMN TO JOINED GAPLC ROWS ### '''
    selSite_merged_df["envData"] = selSite_df.envData.unique()[0]

    ''' ### CONVERT area_allSelSites_km2_y TO ZERO FROM "" ### '''
    ## if sites do not exist (selSitesExist = 0), then make sure area_allSelSites_km2_y = 0 (no selected sites, or empty feature class/shapefile)
    selSite_merged_df.loc[selSite_merged_df.selSitesExist == 0, "area_allSelSites_km2_y"] = 0

    ''' ### CONVERT area_envData_km2 TO ZERO FROM NaN OR BLANK ### '''
    print(selSite_merged_df.area_envData_km2.dtype)
    ## if a column value for "area_envData_km2" is NaN, assign 0 
    selSite_merged_df.loc[(selSite_merged_df['area_envData_km2'].isna()), 'area_envData_km2'] = 0
    ## if a column value for "area_envData_km2" is == "NA", assign 0 (these are runs with empty feature classes or no selected sites) 
    selSite_merged_df.loc[(selSite_merged_df['area_envData_km2'].astype(str) =="NA"), 'area_envData_km2'] = 0
    ## if a column value for "area_envData_km2" is blank/empty, assign to value of area_allSelSites_km2_y  
    ## (for "erased" csvs, if the column is empty, that means there was complete overlap between the sites and the env dataset, so assign area_envData_km2 to area_selSite_km2)
    selSite_merged_df.loc[(selSite_merged_df['area_envData_km2'].astype(str) ==""), 'area_envData_km2'] = selSite_merged_df.area_allSelSites_km2_y
    ## convert column datatype to float 
    selSite_merged_df.area_envData_km2 = pd.to_numeric(selSite_merged_df.area_envData_km2)
    print(selSite_merged_df.area_envData_km2.dtype)

    '''### CREATE NEW COLUMNS: tech, envCat, area_allSelSites_km2 AFTER JOIN ###'''
    selSite_merged_df["area_allSelSites_km2"] = pd.to_numeric(selSite_merged_df["area_allSelSites_km2_y"])
    selSite_merged_df["tech"] = selSite_merged_df["tech_y"] 
    selSite_merged_df["envCat"] = selSite_merged_df["envCat_y"]

    '''### CREATE NEW envCat_scen TO PARSE OUT BaseUSeCat1 AS "Base" ###'''
    selSite_merged_df['envCat_scen'] = np.where(selSite_merged_df.selSites.str.contains("BaseUseCat1"), "Base", selSite_merged_df.envCat) 

    '''### CREATE OR UPDATE FLAG COLUMNS FOR DIFFERENT "ZEROS": impactEval, scenRun ###'''
    ## impactEval: assign value of 0 for all rows that had to be added after the merge
    selSite_merged_df.loc[selSite_merged_df['impactEval'].isna(), 'impactEval'] = 0
    selSite_merged_df.impactEval = selSite_merged_df.impactEval.astype(int) 

    ## scenRun: will add the = 0 flag later when the additional rows are added for scenarios that were not run
    selSite_merged_df["scenRun"] = 1

    ''' ### PARSE OUT CATEGORICAL COLUMNS FROM selSites COLUMN ### '''
    #### geography: InState, Full_WECC, Part_WECc
    selSite_merged_df['geography'] = np.where(selSite_merged_df.selSites.str.contains("InState"), "InState", \
                                              np.where(selSite_merged_df.selSites.str.contains("Full_WECC"), "Full_WECC", \
                                                       np.where(selSite_merged_df.selSites.str.contains("Part_WECC"), "Part_WECC","NA")))

    ## RESOLVE scenario (Capped_Basecase, Capped_highDER, etc.)
    selSite_merged_df['RESOLVE_scenario'] = np.where(selSite_merged_df.selSites.str.contains("Capped_Basecase"), "Capped_Basecase",
                                  np.where(selSite_merged_df.selSites.str.contains("BaseUseCat1_Basecase"), "Capped_Basecase",
                                           np.where(selSite_merged_df.selSites.str.contains("BaseUseCat1_highDER"), "Capped_highDER",
                                                    np.where(selSite_merged_df.selSites.str.contains("BaseUseCat1_lowBatt"), "Capped_lowBatt",
                                                         np.where(selSite_merged_df.selSites.str.contains("Capped_highDER"), "Capped_highDER",
                                                                 np.where(selSite_merged_df.selSites.str.contains("Capped_lowBatt"), "Capped_lowBatt",
                                                                         np.where(selSite_merged_df.selSites.str.contains("No_Cap_Basecase"), "No_Cap_Basecase",
                                                                                 np.where(selSite_merged_df.selSites.str.contains("No_Cap_highDER"), "No_Cap_highDER",
                                                                                         np.where(selSite_merged_df.selSites.str.contains("No_Cap_lowBatt"), "No_Cap_lowBatt", "NA")))))))))

    ### create new "region_final" field for state and RESOLVE Zone compatibility (remove "_%tech% from RESOLVE_zone region names" and add hyphen to New Mexico for No_Cap/W2W rows)
    selSite_merged_df["region_final"] = np.where(selSite_merged_df.region.str.contains("_Wind"), selSite_merged_df.region.str.replace("_Wind", ""), \
                                                  np.where(selSite_merged_df.region.str.contains("_Solar"), selSite_merged_df.region.str.replace("_Solar", ""),\
                                                             np.where(selSite_merged_df.region.str.contains("_Geothermal"), selSite_merged_df.region.str.replace("_Geothermal", ""), selSite_merged_df.region)))
    
    ## Capped vs. No_Cap
    selSite_merged_df['RESOLVE_cap'] = np.where(selSite_merged_df.RESOLVE_scenario.str.contains("Capped"), "Capped",
                                                np.where(selSite_merged_df.RESOLVE_scenario.str.contains("No_Cap"), "No_Cap", selSite_merged_df.RESOLVE_scenario))
    orig_len = len(selSite_merged_df)                                            
   
    ## SET new region columns
    ### region_state: combine all of california, keep OR and WA separate for W2W analyses, rename Southern_Nevada as Nevada
    selSite_merged_df['region_state'] = np.where(selSite_merged_df.selSites.str.contains("CA"), "California",
                                                 np.where(selSite_merged_df.region.str.contains("Southern_Nevada"), "Nevada", 
                                                          np.where(selSite_merged_df.region.str.contains("New Mexico"), "New_Mexico", selSite_merged_df.region_final)))
    
    ### region_state_PNW: like region_state, but names OR And WA as PNW
    selSite_merged_df['region_state_PNW'] = np.where(selSite_merged_df.region.str.contains("Oregon"), "Pacific_Northwest",
                                          np.where(selSite_merged_df.region.str.contains("Washington"), "Pacific_Northwest", selSite_merged_df.region_state))
    
    
    print("Number of PNW rows added: " + str(len(selSite_merged_df) - orig_len))

    ### Calculate km2 of un-impacted land for stacked bar 
    selSite_merged_df["area_unimpactedSelSites_km2"] = pd.to_numeric(selSite_merged_df["area_allSelSites_km2"]) - pd.to_numeric(selSite_merged_df["area_envData_km2"])
 
    return selSite_merged_df

### Function: pre-process housing density and gaplc csv

In [5]:
def processCSV_hd_gaplc(csv, envDataType, valVars = ["0", "553","555", "556", "557"]):
    selSite_df = pd.read_csv(csv, keep_default_na=False)

    ''' ### ADD INFRASTRUCTURE COLUMN ### '''
    if "selSite" in csv:
        selSite_df["infrastructure"] = "generation"
    if "tx" in csv:
        selSite_df["infrastructure"] = "transmission"

    if envDataType == "housingDen":
        ''' ### CONVERT "mean" TO "NA" FROM NaN OR BLANK ### '''
        print(selSite_df["mean"].dtype)
        ## if a column value for "area_envData_km2" is NaN, assign 0 
        selSite_df.loc[(selSite_df['region'].isna()) | (selSite_df['region'].astype(str) =="") | (selSite_df['region'].astype(str) =="NA"), ['mean', 'count']] = math.nan
        ## if a column value for "area_envData_km2" is == "NA", assign 0 (these are runs with empty feature classes or no selected sites) 
        ## convert column datatype to float 
        selSite_df["mean"] = pd.to_numeric(selSite_df["mean"])
        selSite_df["count"] = pd.to_numeric(selSite_df["count"])
        print(selSite_df["mean"].dtype)
        
    if (envDataType == "gaplc") or (envDataType == "rangelands"):
        
        ## Melt the dataframe
        selSite_df = pd.melt(selSite_df, id_vars = ["tech", "envCat", "scenario", "selSites", "region", "selSitesExist", "area_allSelSites_km2"], \
                                           value_vars = valVars).rename(columns= {"variable" : "landCoverType", "value" : "count"})

        ## change "NA" in count column to NaN if selSitesExist == 0
        selSite_df.loc[(selSite_df["count"].astype(str) == "") & (selSite_df.selSitesExist == 0), 'count'] = math.nan
         ## change "" in count column to zeros if selSitesExist == 1
        selSite_df.loc[(selSite_df["count"].astype(str) == "") & (selSite_df.selSitesExist == 1), 'count'] = 0
        selSite_df["count"] = pd.to_numeric(selSite_df["count"])

        ## multiply counts by 30x30m = 0.0009 km2 
        selSite_df["landCoverArea_km2"] = selSite_df["count"]*0.0009

    '''### CREATE NEW envCat_scen TO PARSE OUT BaseUSeCat1 AS "Base" ###'''
    selSite_df['envCat_scen'] = np.where(selSite_df.selSites.str.contains("BaseUseCat1"), "Base", selSite_df.envCat) 

    ''' ### PARSE OUT CATEGORICAL COLUMNS FROM selSites COLUMN ### '''
    #### geography: InState, Full_WECC, Part_WECc
    selSite_df['geography'] = np.where(selSite_df.selSites.str.contains("InState"), "InState", \
                                              np.where(selSite_df.selSites.str.contains("Full_WECC"), "Full_WECC", \
                                                       np.where(selSite_df.selSites.str.contains("Part_WECC"), "Part_WECC","NA")))

    ## RESOLVE scenario (Capped_Basecase, Capped_highDER, etc.)
    selSite_df['RESOLVE_scenario'] = np.where(selSite_df.selSites.str.contains("Capped_Basecase"), "Capped_Basecase",
                                  np.where(selSite_df.selSites.str.contains("BaseUseCat1_Basecase"), "Capped_Basecase",
                                           np.where(selSite_df.selSites.str.contains("BaseUseCat1_highDER"), "Capped_highDER",
                                                    np.where(selSite_df.selSites.str.contains("BaseUseCat1_lowBatt"), "Capped_lowBatt",
                                                         np.where(selSite_df.selSites.str.contains("Capped_highDER"), "Capped_highDER",
                                                                 np.where(selSite_df.selSites.str.contains("Capped_lowBatt"), "Capped_lowBatt",
                                                                         np.where(selSite_df.selSites.str.contains("No_Cap_Basecase"), "No_Cap_Basecase",
                                                                                 np.where(selSite_df.selSites.str.contains("No_Cap_highDER"), "No_Cap_highDER",
                                                                                         np.where(selSite_df.selSites.str.contains("No_Cap_lowBatt"), "No_Cap_lowBatt", "NA")))))))))

    ### create new "region_final" field for state and RESOLVE Zone compatibility (remove "_%tech% from RESOLVE_zone region names" and add hyphen to New Mexico for No_Cap/W2W rows)
    selSite_df["region_final"] = np.where(selSite_df.region.str.contains("_Wind"), selSite_df.region.str.replace("_Wind", ""), \
                                                  np.where(selSite_df.region.str.contains("_Solar"), selSite_df.region.str.replace("_Solar", ""),\
                                                             np.where(selSite_df.region.str.contains("_Geothermal"), selSite_df.region.str.replace("_Geothermal", ""), selSite_df.region)))
    
    ## Capped vs. No_Cap
    selSite_df['RESOLVE_cap'] = np.where(selSite_df.RESOLVE_scenario.str.contains("Capped"), "Capped",
                                                np.where(selSite_df.RESOLVE_scenario.str.contains("No_Cap"), "No_Cap", selSite_df.RESOLVE_scenario))
                                                
    ''' ### COMBINE OR AND WA TO FORM PNW ROWS FOR W2W/STATE SCENARIOS ###''' 
    ## sum area_envData_km2 and area_allSelSites_km2
    orig_len = len(selSite_df)
    
    if envDataType == "housingDen":
        for scenario in selSite_df.selSites.unique():
            subset = selSite_df[(selSite_df["selSites"] == scenario) & ((selSite_df["region_final"] == "Oregon") | (selSite_df["region_final"] == "Washington"))]
            if len(subset) >0:
                ## create new row with updated calculations and new region_final name
                if len(subset[subset["region_final"] == "Oregon"]) >0:
                    newRow = subset[(subset["region_final"] == "Oregon")]
                else:
                    newRow = subset[(subset["region_final"] == "Washington")]

                ## Calculate count weighted averages
                count_total = subset["count"].sum()
                subset["weightedMean"] = (subset["mean"]/count_total)*subset["count"]
                PNW_mean = subset["weightedMean"].sum()

                ## Assign calculated values to newRow
                newRow["mean"] = PNW_mean
                newRow["count"] = count_total
                #newRow["percent_selSites"] = area_envData_km2_tot/area_allSelSites_km2_tot
                newRow["region_final"] = "Pacific_Northwest"

                ## add row to original df (selSite_df)
                selSite_df = pd.concat([selSite_df,newRow], axis = 0, ignore_index = True)

    ## SET new region columns
    ### region_state: combine all of california, keep OR and WA separate for W2W analyses, rename Southern_Nevada as Nevada
    selSite_df['region_state'] = np.where(selSite_df.selSites.str.contains("CA"), "California",
                                                 np.where(selSite_df.region.str.contains("Southern_Nevada"), "Nevada", 
                                                          np.where(selSite_df.region.str.contains("New Mexico"), "New_Mexico", selSite_df.region_final)))
    
    ### region_state_PNW: like region_state, but names OR And WA as PNW
    selSite_df['region_state_PNW'] = np.where(selSite_df.region.str.contains("Oregon"), "Pacific_Northwest",
                                          np.where(selSite_df.region.str.contains("Washington"), "Pacific_Northwest", selSite_df.region_state))
                                          
    if envDataType == "housingDen":
    ## Recalculate the "all" region row for scenarios that have CA and OOS resources (need to take the weighted average of the two "all" regions)
        for tech in selSite_df.tech.unique():
            for cat in selSite_df.envCat_scen.unique():
                for geo in selSite_df.geography.unique():
                    for scen  in selSite_df.RESOLVE_scenario.unique():
                        subset = selSite_df[(selSite_df["tech"] == tech) & (selSite_df["envCat_scen"] == cat) & (selSite_df["geography"] == geo) & \
                                            (selSite_df["RESOLVE_scenario"] == scen) & (selSite_df["region_final"] == "all")]  
                        if len(subset) == 2:
                            ## Calculate count weighted averages
                            count_total = subset["count"].sum()
                            subset["weightedMean"] = (subset["mean"]/count_total)*subset["count"]
                            all_mean = subset["weightedMean"].sum()

                            ## Assign calculated values
                            ## take the region_state == "all" row and assign the "mean" column the new all_mean calculated value that includes CA
                            selSite_df.loc[(selSite_df["tech"] == tech) & (selSite_df["envCat_scen"] == cat) & (selSite_df["geography"] == geo) &\
                                           (selSite_df["RESOLVE_scenario"] == scen) & (selSite_df["region_state"] == "all"), 'mean']  = all_mean
                            ## take the region_state == "all" row and assign the "count" column the new count_total calculated value that includes CA
                            selSite_df.loc[(selSite_df["tech"] == tech) & (selSite_df["envCat_scen"] == cat) & (selSite_df["geography"] == geo) & \
                                           (selSite_df["RESOLVE_scenario"] == scen) & (selSite_df["region_state"] == "all"), "count"] = count_total
                            ## rename the region_state column for California's total, since all the supercrezs in California also have "California" in region_state (don't use the "California" value for making figures)
                            selSite_df.loc[(selSite_df["tech"] == tech) & (selSite_df["envCat_scen"] == cat) & (selSite_df["geography"] == geo) & \
                                           (selSite_df["RESOLVE_scenario"] == scen) & (selSite_df["region_state"] == "California") & \
                                           (selSite_df["region_final"] == "all"), "region_state"] = "California_all"
                        ## rename the region_state column for California's total in the inState scenarios (which don't have a len == 1)
                        elif len(subset) == 1 and subset["region_state"].where(subset.region_state == 'California', np.nan).max():
                            selSite_df.loc[(selSite_df["tech"] == tech) & (selSite_df["envCat_scen"] == cat) & (selSite_df["geography"] == geo) & \
                                           (selSite_df["RESOLVE_scenario"] == scen) & (selSite_df["region_state"] == "California") & \
                                           (selSite_df["region_final"] == "all"), "region_state"] = "California_all"
                            

    print("Number of PNW rows added: " + str(len(selSite_df) - orig_len))

    ### Calculate km2 of un-impacted land for stacked bar 
    #selSite_df["area_unimpactedSelSites_km2"] = pd.to_numeric(selSite_df["area_allSelSites_km2"]) - pd.to_numeric(selSite_df["area_envData_km2"])

    return selSite_df

## Apply processCSV_hd_gaplc function to housing density and gaplc csvs

In [29]:
# ============> CHANGE INPUT <====================
infrastructureType = "tx_longHaul" ## selSite or tx or tx_longHaul
gaplc = 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_' + infrastructureType + '_SG13_gaplc_df.csv'
hd = 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_' + infrastructureType + '_SG12_housingDensity_df.csv'
rangelands = 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_' + infrastructureType + '_SG17_rangelands_df.csv'

### Process housing density

In [6]:
hd_out = processCSV_hd_gaplc(hd, "housingDen")
hd_out.to_csv("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_master_" + infrastructureType +  "_housingDensity_df.csv", index = False)

object
float64
Number of PNW rows added: 2


### Process gaplc 

In [30]:
gaplc_out = processCSV_hd_gaplc(gaplc, "gaplc", valVars = ["0", "553","555", "556", "557"])
gaplc_out.to_csv("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_master_" + infrastructureType +  "_gaplc_df_v2.csv", index = False)

Number of PNW rows added: 0


### Process rangelands

In [31]:
rangelands_out = processCSV_hd_gaplc(rangelands, "rangelands", valVars = ["0", "1","2","3","4","5","11","12","21","22","23","24","31","80","81","82"])
rangelands_out.to_csv("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_master_" + infrastructureType +  "_rangelands_df_v2.csv", index = False)

Number of PNW rows added: 0


## Apply processCSV function to other environmental metrics
### Read list of csvs from envImpactAssessment directory

In [32]:
# ============> CHANGE INPUT <====================
infrastructureType = "tx_longHaul" ## "tx" or "selSite" or "tx_longHaul"

outputFolder = r"C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment"
## get selected sites csvs
selSite_list = glob.glob(os.path.join(outputFolder, "areaImpacted_" + infrastructureType + "_SG*"))
selSite_list

['C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG01_Cat1_Solar_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG01_Cat1_Wind_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG02_Cat2_Geothermal_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG02_Cat2_Solar_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG02_Cat2_Wind_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG03_Cat3_Geothermal_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_SG03_Cat3_Solar_df.csv',
 'C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_tx_longHaul_S

### Read gaplc csv in order to merge area_allSelSites_km2 for all categories to each csv

In [33]:
gaplc = [x for x in selSite_list if "gaplc" in x]
gaplc
gaplc_df = pd.read_csv(gaplc[0], keep_default_na=False)

## Run parsing function for each csv and concat into master_df, write to csv

In [34]:
area_list = []
for csv in selSite_list:
    if not("housingDensity" in csv) and not("gaplc" in csv) and not("rangelands" in csv):
        print("Working on " + csv)
        ## apply function to preprocess csvs 
        merged_df = processCSV(csv, gaplc_df)
        
        ## add merged df to list for concatenating at the end
        area_list.append(merged_df)
        
master_df = pd.concat(area_list)

print(len(master_df))
## write to csv
master_df.to_csv("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_master_" + infrastructureType +  "_df_v3.csv", index = False)

Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG01_Cat1_Solar_df.csv
float64
float64
Number of PNW rows added: 0
Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG01_Cat1_Wind_df.csv
float64
float64
Number of PNW rows added: 0
Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG02_Cat2_Geothermal_df.csv
float64
float64
Number of PNW rows added: 0
Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG02_Cat2_Solar_df.csv
float64
float64
Number of PNW rows added: 0
Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG02_Cat2_Wind_df.csv
float64
float64
Number of PNW rows added: 0
Working on C:\Users\Grace\Documents\TNC_beyond50\PathTo100\envImpactAssessment\areaImpacted_tx_longHaul_SG03_Cat3_Geothermal_df.csv
float64
fl

## Create legend and write to csv
### Explanation of field/column flags for different "zeros"; writes to a master legend csv for interpreting the columns
#### selSitesExist 

1: if the scenario was run and sites were chosen in that scenario and region and tech (these values were assigned BEFORE merging) <br/> 
0: if the scenario was run, but sites were NOT chosen for that scenario, env cat, region and tech

#### impactEval
1: if impact was calculated for this environmental category <br/>
0: if impact WAS NOT calculated for this env cat because we expected ZERO impact (e.g., Cat 3 env datasets were not evaluated against Cat 3 selected sites because we would expect to see zero impact since Cat 3 env datasets were excluded in the Cat 3 suitable sites) <br/>
0 flags are from rows added after merging with gaplc df

#### scenRun
1: if the scenario was run<br/>
0: if the scenario was NOT run (e.g., W2W low-Batt for Cat 1)


In [757]:
master_df.columns

Index(['tech_x', 'envCat_x', 'scenario', 'selSites', 'region', 'envData', 'area_envData_km2', 'area_allSelSites_km2_x', 'percent_selSites', 'impactEval', 'selSitesExist', 'area_allSelSites_km2_y', 'envCat_y', 'tech_y', 'infrastructure', 'area_allSelSites_km2', 'tech', 'envCat', 'envCat_scen', 'scenRun', 'geography', 'RESOLVE_scenario', 'region_final', 'region_state', 'RESOLVE_cap', 'area_unimpactedSelSites_km2'], dtype='object')

In [762]:
legend = pd.DataFrame(data = {'tech_x': ["IGNORE: technology for selected site or transmission. Where blank no area calculations were performed, refer to 'tech' column instead"], \
                                 'envCat_x': ["IGNORE: environmental category scenario for selectedsite or transmission. Where blank no area calculations were performed, refer to 'envCat' column instead"],\
                                 'scenario': ["Geography scenarios + RESOLVE scenario"], \
                                 'selSites': ["Name of selected sites or transmission polygon file"], \
                                 'region' : ["Name of region from selSites file"], \
                                 'envData': ["Name of enviromental metric dataset (e.g., Cat4_SG04 or CriticalHabitat_SG05)"], \
                                 'area_envData_km2': ["Area of environmental metric dataset that overlaps with the selected sites or transmission polygons. This is km2 of 'impact'"],\
                                 'area_allSelSites_km2_x': ["IGNORE: Area of just the selected sites or transmission polygons calculated during the area analyses. Where blank, refer to the 'area_allSelSites_km2' column"],\
                                 'percent_selSites': ["area_envData_km2/area_allSelSites_km2"],\
                                 'impactEval': ["Flag 1: if impact was calculated for this environmental category (pre-join/merge); Flag 0: if impact WAS NOT calculated for this env cat because we expected ZERO impact (post-join/merge). Rows ==0 could have needed impact evaluation (e.g., Cat 1 solar OOS), but were not actually evaluated because no sites existed."],\
                                 'selSitesExist': ["Flag 1: if the scenario was run in RESOLVE and sites were chosen in that scenario, region, and tech; 0: if the RESOLVE scenario was run, but sites were NOT chosen for that scenario, env cat, region and tech (i.e., empty feature class); this field was joined from gaplc)" ], \
                                 'area_allSelSites_km2_y': ["IGNORE: Area of just the selected sites or transmission polygons calculated for the gaplc analyses. Where blank, refer to the 'area_allSelSites_km2' column"], \
                                 'envCat_y': ["IGNORE: environmental category scenario for selected site or transmission. Where blank no area calculations were performed, refer to 'envCat' column instead"], \
                                 'tech_y': ["IGNORE: technology for selected site or transmission. Where blank no area calculations were performed, refer to 'tech' column instead"], \
                                 'infrastructure': ["Generation or Transmission impacts"], \
                                 'area_allSelSites_km2': ["Area of just the selected sites or transmission polygons calculated during the area analyses. Used for calculating percentage or creating 'unimpacted area' stacked bar"], \
                                 'tech': ["technology for selected site or transmission"], \
                                 'envCat' : ["environmental category scenario used for for selected site (spatial disaggregation) or transmission."], \
                                 'envCat_scen' : ["environmental category scenario used for for selected site or transmission--differs from envCat in that the RESOLVE Base scenario is referred to here as 'Base', not 'Cat1' (since Base required the use of Category 1 site suitability polygons for spatial disaggregation)"], \
                                 'scenRun' : ["Flag 1: if the scenario was run in RESOLVE; Flag 0: if the scenario was NOT run in RESOLVE (e.g., W2W low-Batt for Cat 1)"], \
                                 'geography' : ["InState, Full_WECC, or Part_WECC"], \
                                 'RESOLVE_scenario': ["Capped_Basecase, Capped_highDER, Capped_lowBatt, No_Cap_Basecase, No_Cap_highDER, No_Cap_lowBatt"], \
                                 'region_final' : ["Similar to 'region' field, but is not technology specific and contains Pacific_Northwest region for W2W assessment that sums Oregon and Washington's areas. "], \
                                 'region_state': ["Similar to 'region_final' field, but assigns all CA RESOLVE Zones as 'California' and assigns 'Southern_Nevada' to 'Nevada' (state), 'New Mexico' (state) is also renamed 'New_Mexico' "],\
                                 'RESOLVE_cap' : ["Either 'Capped' (RESOLVE Zone for OOS) or 'No_Cap' (states) "],\
                                 'area_unimpactedSelSites_km2': ["Area of selected sites or transmission that does not overlap with env metric dataset: 'area_allSelSites_km2' - 'area_envData_km2'"]}).transpose()

legend.to_csv("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\areaImpacted_master_legend.csv")                    

## Check resulting processed df

In [741]:
master_df.RESOLVE_scenario.unique()

array(['Capped_Basecase', 'Capped_highDER', 'Capped_lowBatt',
       'No_Cap_Basecase', 'No_Cap_highDER', 'No_Cap_lowBatt'],
      dtype=object)

In [ ]:
master_df[(master_df.impactEval == 0) & (master_df.selSitesExist == 1)]

In [710]:
master_df.columns

Index(['tech_x', 'envCat_x', 'scenario', 'selSites', 'region', 'envData', 'area_envData_km2', 'area_allSelSites_km2_x', 'percent_selSites', 'impactEval', 'selSitesExist', 'area_allSelSites_km2_y', 'envCat_y', 'tech_y', 'infrastructure', 'area_allSelSites_km2', 'tech', 'envCat', 'envCat_scen', 'scenRun', 'geography', 'RESOLVE_scenario', 'region_final', 'area_unimpactedSelSites_km2'], dtype='object')